# Prompting and basic Langchain

Importing necessary libraries and installing required packages

In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
import pprint

from IPython.display import display, Markdown

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
# %pip install langchain langchain-openai langchain-groq langchain-community langchain-chroma

This is all for Langchain

In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from pydantic import BaseModel, Field


## A simple LLM-based Chat

For Groq, you need to get first an account and API KEY at https://groq.com/ 

The API KEY should go to the env variables

In [5]:
llm_model = os.environ["OPENAI_MODEL"]
# llm_model = "moonshotai/kimi-k2-instruct"
print(llm_model)

llm = ChatOpenAI(model=llm_model, temperature=0.7)
# llm = ChatGroq(model=llm_model, temperature=0.1)

response = llm.invoke("Tell me a joke about data scientists")
print(response.content)

gpt-4o-mini
Why did the data scientist break up with the statistician?

Because she found him too mean!


Chat models are based on roles and messages. Let's do our first chain with Langchain

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm # Chaining the prompt and the llm call

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)
print(ai_msg.content)

I said, "J'adore la programmation," which means "I love programming" in French.


This is a another typical kind of chain. Here, we use a *zero-shot* mode, as the LLM answers with its own (pretrained) knowledge.

In [7]:
SYSTEM_PROMPT = """You are an experienced software architect that assists a novice developer 
to design a system. """

qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", SYSTEM_PROMPT),
            ("human", "{question}"),
        ]
)

qa_chain = qa_prompt | llm

query = "What are the prons and cons of the Proxy design pattern?"
result = qa_chain.invoke(input=query)
print(result.content)

# display(Markdown(result.content)) # Result in Markdown format


The Proxy design pattern is a structural design pattern that provides an object representing another object. This intermediary (proxy) can control access to the real object, adding an additional layer of functionality, such as lazy initialization, access control, logging, or caching.

Here are some pros and cons of using the Proxy design pattern:

### Pros:

1. **Control Access**: Proxies can control access to the real object, allowing for security features or restrictions on when and how the object can be used.

2. **Lazy Initialization**: Proxies can instantiate the real object only when it is needed, which can save resources and improve performance.

3. **Additional Functionality**: Proxies can add additional behaviors (e.g., logging, caching, or monitoring) without modifying the real object's code.

4. **Decoupling**: The client code does not need to be aware of the real object. This decouples the client from the implementation details of the real object.

5. **Simplified Client In

## Handling Memory (as part of a chat)

The memory is a list of previous (pairs of) messages between the human and the assistant, which provide *context* for the next iteraction.

In [8]:
CONTEXTUALIZED_PROMPT = """Given a chat history and the latest developer's question
    which might reference context in the chat history, formulate a standalone question
    that can be understood without the chat history. Do NOT answer the question,
    just reformulate it if needed and otherwise return it as is."""

contextualized_qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", CONTEXTUALIZED_PROMPT),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{question}"),
        ]
    )

# This is a possible chain to keep (and compress) past interactions
# It's a form of rewriting
contextualized_qa_chain = contextualized_qa_prompt | llm | StrOutputParser()


# A buffer to store messages from user and assistant
chat_history = ChatMessageHistory()
chat_history.add_user_message(query)
chat_history.add_ai_message(result)


query = "Can I combine the pattern with other patterns?"
ai_msg = contextualized_qa_chain.invoke(
    {
        'question': query, 
        'chat_history': chat_history.messages
    }
)
print(ai_msg)

Can the Proxy design pattern be effectively combined with other design patterns, and if so, what are some examples of such combinations?


Let's use the (compressed) history to answer the new question

In [9]:
def contextualized_question(input: dict):
        if input.get("chat_history"):
            return contextualized_qa_chain
        else:
            return input["question"]

# A new QA chain that reuses the previous chain
qa_chain_with_memory = (
         RunnablePassthrough.assign(
            context=contextualized_question | qa_prompt | llm
        )
    )

result = qa_chain_with_memory.invoke(
    {
        'question': query,  
        'chat_history': chat_history.messages
    }
)

display(Markdown(result['context'].content))

Yes, the Proxy design pattern can be combined with other design patterns to enhance functionality, manage complexity, or improve system architecture. Here are some examples of how the Proxy pattern can be combined with other design patterns:

1. **Proxy and Decorator Pattern**:
   - **Example**: You can use a Proxy to control access to an object while also using a Decorator to add additional responsibilities or behaviors to that object. For instance, if you have a `LoggingProxy` that logs access to a service, you could also have a `CachingDecorator` that caches the results of the service calls to improve performance.

2. **Proxy and Singleton Pattern**:
   - **Example**: A Proxy can be used to control access to a Singleton instance. For instance, if you have a `DatabaseConnection` that should only have one instance, you can use a Proxy that manages access to this instance, ensuring that it is created only once and that all access is routed through the Proxy.

3. **Proxy and Factory Pattern**:
   - **Example**: Use a Factory to create Proxy objects instead of the real objects. This is useful in scenarios where you want to add a layer of abstraction, such as creating a Proxy for a remote service. The Factory can instantiate either the real object or its Proxy based on certain conditions, such as whether the application is running in a local or remote environment.

4. **Proxy and Observer Pattern**:
   - **Example**: A Proxy can be used to monitor changes in a subject that it proxies. For example, if you have a `Subject` that sends notifications to observers, you could implement a Proxy that intercepts these notifications to log them or apply additional filtering before they are sent to the observers.

5. **Proxy and Command Pattern**:
   - **Example**: You can use a Proxy to manage command execution. For instance, a `CommandProxy` might handle authentication or authorization before a command is executed. This way, the command's execution logic remains clean and focused, while the Proxy handles the pre-processing.

6. **Proxy and Flyweight Pattern**:
   - **Example**: If you have a system where many objects are similar and require a lot of memory, you can use the Flyweight pattern to share common parts of the objects and a Proxy to manage their instantiation and access. The Proxy can check if a Flyweight instance already exists and return it instead of creating a new one.

7. **Proxy and Strategy Pattern**:
   - **Example**: A Proxy can be used to select different strategies for a service at runtime. For instance, if you have a service that can process data in different ways depending on the user's preferences, a Proxy can encapsulate the logic for choosing the right strategy based on certain conditions.

By combining the Proxy pattern with other design patterns, you can create more flexible and maintainable systems, leveraging the strengths of each pattern to address specific challenges in your architecture.

In [10]:
# How the whole result looks like
pprint.pprint(result)

{'chat_history': [HumanMessage(content='What are the prons and cons of the Proxy design pattern?', additional_kwargs={}, response_metadata={}),
                  AIMessage(content="The Proxy design pattern is a structural design pattern that provides an object representing another object. This intermediary (proxy) can control access to the real object, adding an additional layer of functionality, such as lazy initialization, access control, logging, or caching.\n\nHere are some pros and cons of using the Proxy design pattern:\n\n### Pros:\n\n1. **Control Access**: Proxies can control access to the real object, allowing for security features or restrictions on when and how the object can be used.\n\n2. **Lazy Initialization**: Proxies can instantiate the real object only when it is needed, which can save resources and improve performance.\n\n3. **Additional Functionality**: Proxies can add additional behaviors (e.g., logging, caching, or monitoring) without modifying the real object's c

## Handling Few-shots

In [11]:
zero_shot_prompt = PromptTemplate(
    input_variables=['input'],
    template="""Return the antonym of the input given along with an explanation.
    Input: {input}
    Output:
    Explanation:
    """
)

# Zero-shot chain
zero_shot_chain = zero_shot_prompt | llm

query = 'happy' # 'I am very sad but still have hope'
result = zero_shot_chain.invoke(input=query)
print(result.content)

Output: sad  
Explanation: The antonym of "happy" is "sad." While "happy" refers to a state of joy, contentment, or pleasure, "sad" describes a state of unhappiness, sorrow, or disappointment. These two words represent opposing emotional states.


In [12]:
# Examples of a task for creating antonyms.
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [13]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    Chroma, # The database to store the examples with their embeddings
    # The number of examples to produce.
    k=1,
)

similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Return the antonym of the given input along with an explanation. \n\nExample(s):",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

print(similar_prompt.format(adjective="rainy"))


Return the antonym of the given input along with an explanation. 

Example(s):

Input: sunny
Output: gloomy

Input: rainy
Output:


And let's use the new prompt in a chain

In [14]:
# Few-shot
few_shot_chain = similar_prompt | llm

query = 'rainy' # 'I am very sad but still have hope'
print(similar_prompt.format(adjective=query))
print()

result = few_shot_chain.invoke(input=query)
print(result.content)

Return the antonym of the given input along with an explanation. 

Example(s):

Input: sunny
Output: gloomy

Input: rainy
Output:

Output: dry

Explanation: The term "rainy" refers to weather conditions characterized by precipitation, specifically rain. The antonym "dry" describes conditions that lack moisture, particularly rain, indicating a lack of wetness or precipitation.


We can ask an LLM to generate its response as a JSON object, using the Pydantic framework. 

For example, we can format the antonym output

In [15]:
class FormattedAntonym(BaseModel):
    antonym: str = Field(description="An antonym for the input word or phrase.")
    explanation: str = Field(description="A short explanation of how the antonym was generated")


llm_with_structure = llm.with_structured_output(FormattedAntonym)

few_shot_chain1 = similar_prompt | llm_with_structure
result = few_shot_chain1.invoke(input=query)
result # The Pydantic (object) format

FormattedAntonym(antonym='dry', explanation="The antonym 'dry' is used here as it contrasts with 'rainy', which indicates wet weather. 'Dry' suggests the absence of rain.")

In [16]:
print(result.model_dump_json(indent=2))
# pprint.pprint(result.model_dump()) # This is a dict

{
  "antonym": "dry",
  "explanation": "The antonym 'dry' is used here as it contrasts with 'rainy', which indicates wet weather. 'Dry' suggests the absence of rain."
}


---